In [2]:
# Data
import numpy as np
np.set_printoptions(precision=2, suppress=True)
import pandas as pd

pd.set_option('display.max_rows', 60)

# Models
# from sklearn.linear_model import LogisticRegression 
# from sklearn.linear_model import Perceptron
# from sklearn import svm #Support Vector Machine
# from sklearn.ensemble import RandomForestClassifier 
# from sklearn.neighbors import KNeighborsClassifier #KNN
# from sklearn.naive_bayes import GaussianNB #Naive bayes
# from sklearn.tree import DecisionTreeClassifier #Decision Tree
# from sklearn import metrics #accuracy measure
# from sklearn.metrics import confusion_matrix 
# from sklearn.ensemble import VotingClassifier
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import train_test_split 

import tensorflow as tf
from tensorflow import keras
import keras.backend as K 
tf.compat.v1.enable_eager_execution()

# Model Helpers
# from sklearn.model_selection import KFold 
# from sklearn.model_selection import cross_val_score 
# from sklearn.model_selection import cross_val_predict 
# from sklearn.model_selection import cross_validate
# from sklearn.model_selection import GridSearchCV
# from sklearn import feature_selection
# from sklearn import model_selection
# from sklearn import metrics
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

#Other
import re
from tqdm import tqdm
from keras.utils.vis_utils import plot_model


In [3]:
df = pd.read_csv("../datasets/train_combined.csv", parse_dates=True)
df["date"] = pd.to_datetime(df["date"])
df["event_day"] = df["event_day"].astype(int)
df["day_off"] = df["day_off"].astype(int)

df = df.drop(["id", "city", "state", "cluster", "type"], axis=1)
df.head()

,date,store_nbr,family,sales,onpromotion,dcoilwtico,event_day,day_off
0,2013-01-01,1,AUTOMOTIVE,0.0,0,93.14,1,1
1,2013-01-01,1,BABY CARE,0.0,0,93.14,1,1
2,2013-01-01,1,BEAUTY,0.0,0,93.14,1,1
3,2013-01-01,1,BEVERAGES,0.0,0,93.14,1,1
4,2013-01-01,1,BOOKS,0.0,0,93.14,1,1


In [4]:
scaled_df = df.copy()

feature_scaler = MinMaxScaler()
fearure_cols = ["sales", "onpromotion", "dcoilwtico"]
scaled_df.loc[:, fearure_cols] = feature_scaler.fit_transform(scaled_df[fearure_cols])

label_encoder = LabelEncoder()
scaled_df["family"] = label_encoder.fit_transform(scaled_df["family"])

scaled_df.set_index("date", drop=True, inplace=True)

scaled_df.head()

,store_nbr,family,sales,onpromotion,dcoilwtico,event_day,day_off
date,,,,,,,
2013-01-01,1,0,0.0,0.0,0.792965,1,1
2013-01-01,1,1,0.0,0.0,0.792965,1,1
2013-01-01,1,2,0.0,0.0,0.792965,1,1
2013-01-01,1,3,0.0,0.0,0.792965,1,1
2013-01-01,1,4,0.0,0.0,0.792965,1,1


In [41]:
win_length = 3
batch_size = 1
num_features = 4


# ts_list = []
def refactor_ts(df, 
                win_length = 3,
                batch_size = 2,
                num_features = 4):
    groups = df.groupby(["store_nbr", "family"])

    X_features = []
    X_store_nbrs = []
    X_family_indices = []
    y_targets = []
    num_unique_stores = df["store_nbr"].nunique()
    num_unique_families = df["family"].nunique()

    with tqdm(total=num_unique_stores * num_unique_families) as pbar:
        for name, group in groups:
            store_nbr, family = name
            pbar.set_description(f"store: {store_nbr}, family: {family}")
            y_group= group["sales"].to_numpy()
            X_group = group.drop(["store_nbr", "family", "sales"], axis=1).to_numpy()
            ts_generator = TimeseriesGenerator(X_group, y_group, 
                                            length=win_length,
                                            sampling_rate=1,
                                            batch_size=batch_size)
            # ts_list.extend([(np.array([store_nbr, family]), X[0], y) for X, y in ts_generator])
                           
            for X, y in ts_generator:
                print(y)
                X_features.append(X)
                X_store_nbrs.append(store_nbr)
                X_family_indices.append(family)
                y_targets.append(y)
                break
            break  
            pbar.update(1)
    X_features = np.array(X_features)
    X_store_nbrs = np.array(X_store_nbrs)
    X_family_indices = np.array(X_family_indices)
    y_targets = np.array(y_targets)
    return (X_features, X_store_nbrs, X_family_indices, y_targets)

X_features, X_store_nbrs, X_family_indices, y_targets = refactor_ts(scaled_df)
    

store: 1, family: 0:   0%|          | 0/1782 [00:00<?, ?it/s]

[0. 0.]


In [39]:
num_unique_stores = df["store_nbr"].nunique()
num_unique_families = df["family"].nunique()

embed_store_dim = 30
inp_embedding_store = keras.layers.Input(shape=(1,))
out_store = keras.layers.Embedding(num_unique_stores + 1, 
                                   embed_store_dim, 
                                   name="store_embedding")(inp_embedding_store)
out_store = keras.layers.SpatialDropout1D(0.3)(out_store)
out_store = keras.layers.Reshape(target_shape=(embed_store_dim,))(out_store)

embed_family_dim = 20
inp_embedding_family = keras.layers.Input(shape=(1,))
out_family = keras.layers.Embedding(num_unique_families + 1, 
                                   embed_family_dim, 
                                   name="family_embedding")(inp_embedding_family)
out_family = keras.layers.SpatialDropout1D(0.3)(out_family)
out_family = keras.layers.Reshape(target_shape=(embed_family_dim,))(out_family)

# out_combined = keras.layers.Concatenate(axis=1)([out_store, out_family])

input_main = keras.layers.Input(shape=(win_length, num_features))

def combine(x):
    input_main, out_store, out_family = x
    print(input_main)
    out_combined = tf.concat([out_store, out_family], axis=1)
    
    repeated_out_combined = tf.broadcast_to(out_combined,(input_main.shape[1],out_combined.shape[1]))
    repeated_out_combined = tf.reshape(repeated_out_combined, [-1, repeated_out_combined.shape[0], repeated_out_combined.shape[1]])
    
    input_main = tf.cast(input_main, tf.float32) 
    repeated_out_combined = tf.cast(repeated_out_combined, tf.float32)     
    return tf.concat([input_main, repeated_out_combined],axis=2)

out = keras.layers.Lambda(combine)([input_main, out_store, out_family])

out = keras.layers.LSTM(128, activation="relu", return_sequences=True)(out)
out = keras.layers.LSTM(64, activation="relu")(out)
out = keras.layers.Dropout(0.2)(out)
out = keras.layers.Dense(1, activation="relu")(out)

model = keras.Model(inputs=[input_main, inp_embedding_store, inp_embedding_family],
                    outputs=out)
model.compile(optimizer="adam", loss="mse")

Tensor("Placeholder:0", shape=(None, 3, 4), dtype=float32)


In [28]:
X_features[320000].shape, X_store_nbrs[320000], X_family_indices[320000], y_targets[320000]

((1, 3, 4), 6, 25, array([0.]))

In [35]:
def combine(x):
    input_main, out_store, out_family = x
    print(out_store, out_family)
    out_combined = tf.concat([out_store, out_family], axis=1)
    
    repeated_out_combined = tf.broadcast_to(out_combined,(input_main.shape[1],out_combined.shape[1]))
    repeated_out_combined = tf.reshape(repeated_out_combined, [-1, repeated_out_combined.shape[0], repeated_out_combined.shape[1]])
    
    input_main = tf.cast(input_main, tf.float32) 
    repeated_out_combined = tf.cast(repeated_out_combined, tf.float32)   
    print(repeated_out_combined)  
    return tf.concat([input_main, repeated_out_combined],axis=2)

combine([tf.Variable(X_features[320000]),
         tf.Variable([[0.1, 0.3, 0.6]]), 
         tf.Variable([[0.8, 0.7]])])

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float32, numpy=array([[0.1, 0.3, 0.6]], dtype=float32)> <tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[0.8, 0.7]], dtype=float32)>
tf.Tensor(
[[[0.1 0.3 0.6 0.8 0.7]
  [0.1 0.3 0.6 0.8 0.7]
  [0.1 0.3 0.6 0.8 0.7]]], shape=(1, 3, 5), dtype=float32)


<tf.Tensor: shape=(1, 3, 9), dtype=float32, numpy=
array([[[0.  , 0.81, 0.  , 0.  , 0.1 , 0.3 , 0.6 , 0.8 , 0.7 ],
        [0.  , 0.8 , 0.  , 0.  , 0.1 , 0.3 , 0.6 , 0.8 , 0.7 ],
        [0.  , 0.79, 0.  , 1.  , 0.1 , 0.3 , 0.6 , 0.8 , 0.7 ]]],
      dtype=float32)>

In [30]:
tf.Variable(X_features[320000])

<tf.Variable 'Variable:0' shape=(1, 3, 4) dtype=float64, numpy=
array([[[0.  , 0.81, 0.  , 0.  ],
        [0.  , 0.8 , 0.  , 0.  ],
        [0.  , 0.79, 0.  , 1.  ]]])>

In [36]:
model.fit([X_features, X_store_nbrs, X_family_indices], y_targets,
          epochs=1, batch_size=1, validation_split=0.1, verbose=1)

ValueError: in user code:

    File "c:\Users\Cyril\anaconda3\envs\DataScience3_9\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Cyril\anaconda3\envs\DataScience3_9\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Cyril\anaconda3\envs\DataScience3_9\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Cyril\anaconda3\envs\DataScience3_9\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Cyril\anaconda3\envs\DataScience3_9\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Cyril\anaconda3\envs\DataScience3_9\lib\site-packages\k